# Goal = properly understanding decorators and wrappers

1. **Wrapper decorators** - replace the original function with a new wrapped version
2. **Registration decorators** - register functions but leave them unchanged



In [ ]:
# for performance logger

# just two placeholders that do nothing
def some_function(*args): pass
def time_function(*args): pass
# What happens behind @time_function("test", "API")
original_function = some_function
# Step 1: Call factory
decorator = time_function("test", "API")  
# Step 2: apply decorator  
wrapped_function = decorator(original_function)
# Step 3: replace original
some_function = wrapped_function  # now including new logic
# => actually creates a new function to be called! 

In [ ]:
# for event bus

# empty placeholders that do nothing (just for the interpreter to be happy)
def some_function(): pass
def events(self):
    def on(): pass
# what goes on behind @events.on('some_function')
original_function = some_function
# Step 1: call method
decorator = events.on('user_login')
# Step 2: apply decorator
result_function = decorator(original_function)  # registers + returns original
# Step 3: function unchanged
some_function = result_function  # still same function (as opposed to how it worked with decorator->wrapper)
# => it registers the function in the eventbus to be called later, but doesn't change the function

## 1. Basic Decorator Implementation

Let's start with a simple decorator to understand the basic pattern:

In [5]:
def simple_decorator(func):
    """A basic decorator that wraps a function"""
    def wrapper(*args, **kwargs):
        print(f"Before calling {func.__name__}")
        result = func(*args, **kwargs) 
        print(f"After calling {func.__name__}")
        return result
    return wrapper

# Test function
def greet(name):
    print(f"Hello, {name}!")
    return f"Greeting for {name}" 

# Manual decoration (what @ does behind the scenes)
print("=== Manual decoration ===")
original_greet = greet
decorated_greet = simple_decorator(greet)

print("Original function:")
result1 = original_greet("Alice")
print(f"Return value by original function: {result1}\n")

print("Decorated function:")
result2 = decorated_greet("Bob")
print(f"Return value of original, but handed through decorated function: {result2}")

=== Manual decoration ===
Original function:
Hello, Alice!
Return value by original function: Greeting for Alice

Decorated function:
Before calling greet
Hello, Bob!
After calling greet
Return value of original, but handed through decorated function: Greeting for Bob


Output: 

=== Manual decoration ===       // 
Original function:              //  
Hello, Alice!                   // from greet
Return value of original function: Greeting for Alice

Decorated function:             // 
Before calling greet            // from wrapper
Hello, Bob!                     // from greet called inside wrapper
After calling greet             // from wrapper
Return value of original, but handed through decorated function: Greeting for Bob

## 2. Decorator Factory with Parameters

Now let's implement a decorator factory that accepts parameters (like your `time_function("test", "API")`):

In [ ]:
import time
import functools

def time_function(category="", api_type=""):
    """Decorator factory that creates timing decorators with parameters"""
    def decorator(func):
        @functools.wraps(func)  # Preserves original function metadata
        def wrapper(*args, **kwargs):
            start_time = time.time()
            print(f"[{category}] Starting {api_type} call to {func.__name__}") # has access to variables from parent time_function
            
            result = func(*args, **kwargs) # executes the wrapped original function with the arguments it was called with
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"[{category}] {api_type} call to {func.__name__} took {duration:.4f} seconds")
            return result
        return wrapper
    return decorator




# Test this step by step
def api_call(endpoint):
    """Simulate an API call"""
    time.sleep(0.1)  # Simulate network delay
    return f"<Data from {endpoint}>"

print("=== Step-by-step decoration process ===")
original_function = api_call

# Step 1: Call factory to get decorator
print("Step 1: Creating decorator...")
decorator = time_function("test", "API")
print(f"Decorator created: {decorator}")

# Step 2: Apply decorator to get wrapped function
print("\nStep 2: Applying decorator...")
wrapped_function = decorator(original_function)
print(f"Wrapped function created: {wrapped_function}")

# Step 3: Replace original (this is what @ does)
print("\nStep 3: Testing wrapped function...")
result = wrapped_function("/users")
print(f"Result: {result}")



## As it would really be used
@time_function("test2", "API")
def api_call2(endpoint):
    """Simulate an API call"""
    time.sleep(0.1)  # Simulate network delay
    return f"<Data from {endpoint}>"

print("\nAs it would really be used: ")
result = api_call2('/users')
print(f"Result: {result}")




## so... 
print("\nThe equivalent if we manual called it: ")

# the arguments that were passed in above
category = "test"
api_type = "API"
func_name = "api_call"

# same as the insides of wrapper
start_time = time.time()
print(f"[{category}] Starting {api_type} call to {func_name}") 

# the decorated function function just as it would be called inside wrapper
result = api_call("/users")

end_time = time.time()
duration = end_time - start_time
print(f"[{category}] {api_type} call to {func_name} took {duration:.4f} seconds")

print(f"Result: {result}")


=== Step-by-step decoration process ===
Step 1: Creating decorator...
Decorator created: <function time_function.<locals>.decorator at 0x000002AF581A3380>

Step 2: Applying decorator...
Wrapped function created: <function api_call at 0x000002AF581A2980>

Step 3: Testing wrapped function...
[test] Starting API call to api_call
[test] API call to api_call took 0.1003 seconds
Result: <Data from /users>

As it would really be used: 
[test2] Starting API call to api_call2
[test2] API call to api_call2 took 0.1004 seconds
Result: <Data from /users>

The equivalent if we manual called it: 
[test] Starting API call to api_call
[test] API call to api_call took 0.1005 seconds
Result: <Data from /users>


## 3. Function Wrapper with Timing Logic

Let's see the complete timing decorator in action with the `@` syntax:

In [17]:
# Using the @ syntax (equivalent to the manual process above)
@time_function("production", "REST")
def fetch_user_data(user_id):
    """Fetch user data from API"""
    time.sleep(0.05)  # Simulate API call
    return {"id": user_id, "name": f"User {user_id}", "active": True}

@time_function("production", "GraphQL") 
def fetch_posts(user_id, limit=10):
    """Fetch user posts"""
    time.sleep(0.03)  # Simulate API call
    return [f"Post {i} by user {user_id}" for i in range(limit)]

print("=== Testing wrapped functions ===")
user = fetch_user_data(123)
print(f"User data: {user}\n")

posts = fetch_posts(123, limit=3)
print(f"Posts: {posts}")

# The original function is completely replaced!
print(f"\nFunction name: {fetch_user_data.__name__}")
print(f"Function type: {type(fetch_user_data)}")

=== Testing wrapped functions ===
[production] Starting REST call to fetch_user_data
[production] REST call to fetch_user_data took 0.0502 seconds
User data: {'id': 123, 'name': 'User 123', 'active': True}

[production] Starting GraphQL call to fetch_posts
[production] GraphQL call to fetch_posts took 0.0304 seconds
Posts: ['Post 0 by user 123', 'Post 1 by user 123', 'Post 2 by user 123']

Function name: fetch_user_data
Function type: <class 'function'>


# 3.1 How to disable it conditionally

In [ ]:
# 1. Conditional Decorator Factory
def time_function1(category="", api_type="", enabled=True):
    """Decorator factory with conditional timing"""
    def decorator(func):
        if not enabled:
            # Return original function unchanged
            return func
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            print(f"[{category}] Starting {api_type} call to {func.__name__}")
            
            result = func(*args, **kwargs)
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"[{category}] {api_type} call to {func.__name__} took {duration:.4f} seconds")
            return result
        return wrapper
    return decorator

# Usage
DEBUG_MODE = False

@time_function1("production", "REST", enabled=DEBUG_MODE)
def fetch_user_data(user_id):
    time.sleep(0.05)
    return {"id": user_id, "name": f"User {user_id}"}

# Test
print("Testing with DEBUG_MODE =", DEBUG_MODE)
result = fetch_user_data(123)
print(f"Result: {result}")



## 2. Environment-based Conditional
import os

def time_function2(category="", api_type=""):
    """Decorator that checks environment variables"""
    def decorator(func):
        # Check environment or config
        timing_enabled = os.getenv('ENABLE_TIMING', 'false').lower() == 'true'
        
        if not timing_enabled:
            return func  # Return original function
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            print(f"[{category}] Starting {api_type} call to {func.__name__}")
            
            result = func(*args, **kwargs)
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"[{category}] {api_type} call to {func.__name__} took {duration:.4f} seconds")
            return result
        return wrapper
    return decorator

# Set environment variable to test
os.environ['ENABLE_TIMING'] = 'false'  # or 'true'

@time_function2("production", "REST")
def fetch_user_data(user_id):
    time.sleep(0.05)
    return {"id": user_id, "name": f"User {user_id}"}


# 3. Global Toggle Pattern

# Global configuration
class Config:
    TIMING_ENABLED = True
    DEBUG_MODE = False

def time_function3(category="", api_type=""):
    """Decorator that checks global config"""
    def decorator(func):
        if not Config.TIMING_ENABLED:
            return func
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            print(f"[{category}] Starting {api_type} call to {func.__name__}")
            
            result = func(*args, **kwargs)
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"[{category}] {api_type} call to {func.__name__} took {duration:.4f} seconds")
            return result
        return wrapper
    return decorator

# Toggle timing on/off
Config.TIMING_ENABLED = False

@time_function3("production", "REST")
def fetch_user_data(user_id):
    time.sleep(0.05)
    return {"id": user_id, "name": f"User {user_id}"}




# 4. Conditional Decorator Helper

def conditional_decorator(condition, decorator_factory):
    """Helper to conditionally apply decorators"""
    def apply_if_true(*args, **kwargs):
        if condition:
            return decorator_factory(*args, **kwargs)
        else:
            # Return a no-op decorator
            return lambda func: func
    return apply_if_true

# Usage
NO_TIMING = True

# Create conditional version
conditional_timer = conditional_decorator(not NO_TIMING, time_function)

@conditional_timer("production", "REST")
def fetch_user_data(user_id):
    time.sleep(0.05)
    return {"id": user_id, "name": f"User {user_id}"}



# 5. Runtime Toggle Pattern

class TimingDecorator:
    """Decorator class that can be toggled at runtime"""
    
    def __init__(self, enabled=True):
        self.enabled = enabled
    
    def __call__(self, category="", api_type=""):
        def decorator(func):
            @functools.wraps(func)
            def wrapper(*args, **kwargs):
                if not self.enabled:
                    # Skip timing, just call function
                    return func(*args, **kwargs)
                
                start_time = time.time()
                print(f"[{category}] Starting {api_type} call to {func.__name__}")
                
                result = func(*args, **kwargs)
                
                end_time = time.time()
                duration = end_time - start_time
                print(f"[{category}] {api_type} call to {func.__name__} took {duration:.4f} seconds")
                return result
            return wrapper
        return decorator
    
    def enable(self):
        self.enabled = True
    
    def disable(self):
        self.enabled = False

# Usage
timer = TimingDecorator(enabled=False)

@timer("production", "REST")
def fetch_user_data(user_id):
    time.sleep(0.05)
    return {"id": user_id, "name": f"User {user_id}"}

# Can toggle at runtime
print("First call (timing disabled):")
fetch_user_data(123)

timer.enable()
print("\nSecond call (timing enabled):")
fetch_user_data(456)



# All patterns tested


# Test all conditional patterns
print("=== CONDITIONAL DECORATOR PATTERNS ===\n")

# Pattern 1: Parameter-based
print("1. Parameter-based conditional:")
@time_function1("test", "API", enabled=False)
def test1():
    time.sleep(0.1)
    return "test1 result"

result = test1()
print(f"Result: {result}\n")


# Pattern 2: Environment-based
print("2. Environment-based conditional:")

os.environ['ENABLE_TIMING'] = 'false'
print(f"os.environ['ENABLE_TIMING' = 'false'")
@time_function2("test", "API")
def test2():
    time.sleep(0.1)
    return "test2 result"

result = test2()
print(f"Result: {result}\n")


os.environ['ENABLE_TIMING'] = 'true'  # or 'true'
print(f"os.environ['ENABLE_TIMING' = 'true'")
@time_function2("test", "API")
def test2():
    time.sleep(0.1)
    return "test2 result"

result = test2()
print(f"Result: {result}\n")




# Pattern 3: Global config
print("3. Global config conditional:")

print("Global config disabled")
Config.TIMING_ENABLED = False
@time_function3("test", "API")  # Uses global config version
def test3():
    time.sleep(0.1)
    return "test3 result"

result = test3()
print(f"Result: {result}\n")

Config.TIMING_ENABLED = True
print("Global config enabled")
@time_function3("test", "API")  # Uses global config version
def test3():
    time.sleep(0.1)
    return "test3 result"

result = test3()
print(f"Result: {result}\n")

# Pattern 4: decorator helper (don't like it)
print("4. conditional decorator:")

NO_TIMING = False # NEEDS to be above the next line
conditional_timer = conditional_decorator(not NO_TIMING, time_function)

@conditional_timer()
def test4():
    time.sleep(0.1)
    return "test4 result"
result = test4()
print(f"Result: {result}\n")

# Pattern 5: Runtime toggle
print("5. Runtime toggle:")
timer = TimingDecorator(enabled=False)

@timer("test", "API")
def test5():
    time.sleep(0.1)
    return "test5 result"

print("Disabled:")
result = test5()
print(f"Result: {result}")

timer.enable()
print("Enabled:")
result = test5()
print(f"Result: {result}")

Testing with DEBUG_MODE = False
Result: {'id': 123, 'name': 'User 123'}
First call (timing disabled):

Second call (timing enabled):
[production] Starting REST call to fetch_user_data
[production] REST call to fetch_user_data took 0.0504 seconds
=== CONDITIONAL DECORATOR PATTERNS ===

1. Parameter-based conditional:
Result: test1 result

2. Environment-based conditional:
os.environ['ENABLE_TIMING' = 'false'
Result: test2 result

os.environ['ENABLE_TIMING' = 'true'
[test] Starting API call to test2
[test] API call to test2 took 0.1007 seconds
Result: test2 result

3. Global config conditional:
Global config disabled
Result: test3 result

Global config enabled
[test] Starting API call to test3
[test] API call to test3 took 0.1002 seconds
Result: test3 result

4. conditional decorator:
[] Starting  call to test4
[]  call to test4 took 0.1003 seconds
Result: test4 result

5. Runtime toggle:
Disabled:
Result: test5 result
Enabled:
[test] Starting API call to test5
[test] API call to test5 t

## 4. Event Registration Decorator

Now let's create an event registration system that registers functions but doesn't wrap them:

In [ ]:
class EventBus:
    """Simple event bus for registering and triggering events"""
    
    def __init__(self):
        self.handlers = {}
    
    def on(self, event_name):
        """Decorator that registers a function for an event"""
        def decorator(func):
            # Register the function
            if event_name not in self.handlers:
                self.handlers[event_name] = []
            self.handlers[event_name].append(func)
            print(f"Registered {func.__name__} for event '{event_name}'")
            
            # Return the ORIGINAL function unchanged
            return func
        return decorator
    
    def trigger(self, event_name, *args, **kwargs):
        """Trigger all handlers for an event"""
        if event_name in self.handlers:
            print(f"\nTriggering event '{event_name}':")
            for handler in self.handlers[event_name]:
                try:
                    result = handler(*args, **kwargs)
                    print(f"  {handler.__name__} -> {result}")
                except Exception as e:
                    print(f"  {handler.__name__} failed: {e}")
        else:
            print(f"No handlers for event '{event_name}'")
    
    def list_events(self):
        """Show all registered events"""
        for event, handlers in self.handlers.items():
            handler_names = [h.__name__ for h in handlers]
            print(f"Event '{event}': {handler_names}")

# Create event bus instance
events = EventBus()

# Let's trace what happens step by step
def user_login(username):
    return f"Welcome {username}!"

print("=== Step-by-step event registration ===")
original_function = user_login

# Step 1: Call method to get decorator
print("Step 1: Getting decorator...")
decorator = events.on('user_login')
print(f"Decorator: {decorator}")

# Step 2: Apply decorator
print("\nStep 2: Applying decorator...")
result_function = decorator(original_function)

# Step 3: Check if function changed
print("\nStep 3: Checking if function changed...")
print(f"Original function: {original_function}")
print(f"Result function: {result_function}")
print(f"Are they the same object? {original_function is result_function}")

print(type(events.handlers)) # dict

=== Step-by-step event registration ===
Step 1: Getting decorator...
Decorator: <function EventBus.on.<locals>.decorator at 0x000002AF58689300>

Step 2: Applying decorator...
Registered user_login for event 'user_login'

Step 3: Checking if function changed...
Original function: <function user_login at 0x000002AF586884A0>
Result function: <function user_login at 0x000002AF586884A0>
Are they the same object? True
<class 'dict'>


In [80]:
# The event decorator is essentially a fancy, 
# more organized way of collecting functions into lists and calling them later based on conditions.
# or rather, it uses generally dictionaries to link them to events, but we can do that with conditions, too

def f1(): print("Function 1")
def f2(): print("Function 2") 
def f3(): print("Function 3")
def f4(): print("I'm new!")

funcs = [f1, f2, f3]  # Manual registration
# to add or remove functions (register or deregister them), simply add or remove them from the list
funcs.append(f4)

some_condition = True

# calls the functions when the condition is met
# could have a whole bunch of conditions and respective functions to call
# the conditions could then be set by other parts of the code or inside the executor or its functions
# -> essentially the same as events 
def executor():
    if some_condition:
        for f in funcs:
            f()
    time.sleep(1) # if we moved this to the loop, it would be an event_handler

# just a little loop to showcase it 
# would be an endless loop in practise
# i = 10
# while i > 0:
#     if int(time.time()) % 3 != 0: some_condition = False
#     else: some_condition = True
#     print (int(time.time()))
#     executor()
#     i -= 1
    
    

conditions_and_functions = {
    "con1": {"active": False, "functions": [f1, f2]},
    "con2": {"active": True, "functions": [f2]},
    "con3": {"active": False, "functions": [f1, f3]}, 
    "con4": {"active": False, "functions": [f2, f4]}
}

print(conditions_and_functions)


def dict_executor(some_dict):
    
    for condition_name, condition_data in some_dict.items(): # iterate through both keys and values of outer dict
        if condition_data["active"] is True:
            print(f"Executing {condition_name}")
            for f in condition_data["functions"]: # iterate through values of inner dict
                f()
    
    # same thing (not really practical, just for me to get used to weird python loops and data types)
    for con in some_dict: # iterate through outer keys
        if con[0] is True: # value of first key
            for f in some_dict[con[1]]: # iterate over list (value of 2nd key)
                f()

dict_executor(conditions_and_functions)

conALL = True
conditions_and_functions["conALL"] = {"active" : True, "functions" : [f1,f2,f3,f4]}
print(conditions_and_functions)

dict_executor(conditions_and_functions)
    

{'con1': {'active': False, 'functions': [<function f1 at 0x000002AF5986A8E0>, <function f2 at 0x000002AF598691C0>]}, 'con2': {'active': True, 'functions': [<function f2 at 0x000002AF598691C0>]}, 'con3': {'active': False, 'functions': [<function f1 at 0x000002AF5986A8E0>, <function f3 at 0x000002AF59868860>]}, 'con4': {'active': False, 'functions': [<function f2 at 0x000002AF598691C0>, <function f4 at 0x000002AF5986A5C0>]}}
Executing con2
Function 2
{'con1': {'active': False, 'functions': [<function f1 at 0x000002AF5986A8E0>, <function f2 at 0x000002AF598691C0>]}, 'con2': {'active': True, 'functions': [<function f2 at 0x000002AF598691C0>]}, 'con3': {'active': False, 'functions': [<function f1 at 0x000002AF5986A8E0>, <function f3 at 0x000002AF59868860>]}, 'con4': {'active': False, 'functions': [<function f2 at 0x000002AF598691C0>, <function f4 at 0x000002AF5986A5C0>]}, 'conALL': {'active': True, 'functions': [<function f1 at 0x000002AF5986A8E0>, <function f2 at 0x000002AF598691C0>, <func

In [ ]:
# from copilot:

# What the EventBus Adds

# The decorator pattern just gives you:
# Cleaner syntax - @events.on('startup') vs funcs.append(f1)
# Categorization - Different event names for different function groups
# Error handling - Try/catch around each function call
# Metadata - Function names in output
# Flexibility - Easy to add/remove handlers

# Your Mental Model is Perfect!

username = "user1"

# Your version
user_login_funcs = []
user_logout_funcs = []
error_funcs = []

def handle_user_login(): pass
def log_user_login(): pass
def send_welcome_email(): pass

user_login_funcs.extend([handle_user_login, log_user_login, send_welcome_email])

# When user logs in:
for func in user_login_funcs:
    func(username)

# EventBus version
@events.on('user_login')
def handle_user_login(): pass

@events.on('user_login') 
def log_user_login(): pass

@events.on('user_login')
def send_welcome_email(): pass

# When user logs in:
events.trigger('user_login', username)



# It's All About Organization
# Both approaches:

# ✅ Store functions in lists/collections
# ✅ Call them later based on conditions
# ✅ Allow multiple functions per event/condition
# The decorator just makes it prettier and more organized. Under the hood, it's exactly what you described - fancy function list management!

# You've understood the core concept perfectly. The rest is just syntactic sugar! 🍭

TypeError: handle_user_login() takes 0 positional arguments but 1 was given

## 5. Comparing Wrapper vs Registration Patterns

Let's see both patterns in action and compare their behavior:

In [44]:
# Registration pattern - function stays the same
@events.on('user_logout')
def handle_logout(username):
    return f"Goodbye {username}! Session ended."

@events.on('user_logout')  # Multiple handlers for same event
def log_logout(username):
    return f"LOG: User {username} logged out at {time.time()}"

# Wrapper pattern - function gets replaced
@time_function("auth", "SESSION")
def process_logout(username):
    time.sleep(0.02)  # Simulate processing
    return f"Logout processed for {username}"

print("=== Comparing the patterns ===")

print("\n1. Event registration (function unchanged):")
print(f"handle_logout type: {type(handle_logout)}")
print(f"handle_logout name: {handle_logout.__name__}")

# Call the function directly - it's still the original
direct_result = handle_logout("Alice")
print(f"Direct call result: {direct_result}")

# Trigger through event system
events.trigger('user_logout', "Bob")

print(f"\n2. Wrapper pattern (function replaced):")
print(f"process_logout type: {type(process_logout)}")
print(f"process_logout name: {process_logout.__name__}")

# This calls the wrapped version with timing
wrapped_result = process_logout("Charlie")
print(f"Wrapped call result: {wrapped_result}")

print(f"\n3. Event registry contents:")
events.list_events()

Registered handle_logout for event 'user_logout'
Registered log_logout for event 'user_logout'
=== Comparing the patterns ===

1. Event registration (function unchanged):
handle_logout type: <class 'function'>
handle_logout name: handle_logout
Direct call result: Goodbye Alice! Session ended.

Triggering event 'user_logout':
  handle_logout -> Goodbye Bob! Session ended.
  log_logout -> LOG: User Bob logged out at 1756492343.6125824

2. Wrapper pattern (function replaced):
process_logout type: <class 'function'>
process_logout name: process_logout
[auth] Starting SESSION call to process_logout
[auth] SESSION call to process_logout took 0.0204 seconds
Wrapped call result: Logout processed for Charlie

3. Event registry contents:
Event 'user_login': ['user_login']
Event 'user_logout': ['handle_logout', 'log_logout']


## 6. Testing Decorator Behavior

Let's create a comprehensive test to show the key differences:

In [ ]:
def test_decorator_differences():
    """Comprehensive test showing decorator behavior differences"""
    
    print("=" * 60)
    print("DECORATOR BEHAVIOR COMPARISON")
    print("=" * 60)
    
    # Create a fresh event bus for clean testing
    test_events = EventBus()
    
    # Test 1: Registration decorator
    print("\n1. REGISTRATION DECORATOR TEST")
    print("-" * 40)
    
    @test_events.on('test_event')
    def original_func(x):
        return x * 2
    
    print(f"Function identity preserved: {original_func.__name__}")
    print(f"Direct call: original_func(5) = {original_func(5)}")
    print("Event trigger:")
    test_events.trigger('test_event', 5)
    
    # Test 2: Wrapper decorator  
    print(f"\n2. WRAPPER DECORATOR TEST")
    print("-" * 40)
    
    @time_function("test", "MATH")
    def wrapped_func(x):
        return x * 2
    
    print(f"Function replaced with wrapper: {wrapped_func.__name__}")
    print("Direct call (goes through wrapper):")
    result = wrapped_func(5)
    print(f"Result: {result}")
    
    # Test 3: Multiple registrations
    print(f"\n3. MULTIPLE EVENT HANDLERS")
    print("-" * 40)
    
    @test_events.on('multi_event')
    def handler1(msg):
        return f"Handler 1: {msg}"
    
    @test_events.on('multi_event')
    def handler2(msg):
        return f"Handler 2: {msg.upper()}"
    
    test_events.trigger('multi_event', "hello")
    
    # Test 4: Decorator stacking
    print(f"\n4. STACKING DECORATORS")
    print("-" * 40)
    
    @time_function("test", "STACK")
    @test_events.on('stacked_event')
    def stacked_func(x):
        return f"Processed: {x}"
    
    print("Direct call (wrapped version):")
    direct = stacked_func("test")
    print(f"Result: {direct}")
    
    print("\nEvent trigger (original version):")
    test_events.trigger('stacked_event', "test")
    
    print(f"\n5. SUMMARY")
    print("-" * 40)
    print("Registration decorator: Keeps original function, adds to registry")
    print("Wrapper decorator: Replaces function with enhanced version")
    print("Both can be combined for different behaviors!")

# Run the comprehensive test
test_decorator_differences()